In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [1]:
# get files from drive and install dependencies
# May require runtime restart in colab to properly update packages

%%shell
cp /content/drive/'My Drive'/project.zip /content/
unzip /content//project.zip -d ./project
cd /content/project
pip install -r ./requirements_descriptor_generator.txt

Archive:  /content//project.zip
replace ./project/project_utils.py? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
Looking in links: https://download.pytorch.org/whl/torch_stable.html


In [2]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [0]:
import os
import shutil
import numpy as np
import tensorflow as tf
import torch
import torch.utils.data as tdata
import torch.nn as nn
import torch.nn.parallel
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torchvision import transforms, datasets

os.chdir('./project')

In [0]:
from pointnet_train import training_loop
from pointnet_model import PointNetCls
from matching_cascade import MatchingCascade
from point_cloud_visualization import TrackScatterPlotter
from data_loader import *
import project_utils

In [0]:
# Get TP detections from train_0003 and FP detection from frustum

%%shell

cp /content/drive/'My Drive'/PointNet/Data_folder_train_0003_tracks/zipped_Data_folder_train_0003_tracks.zip /content/train_0003.zip
unzip /content/train_0003.zip -d /content/train_0003

zip_foldername=FP_detections_v3

if [[ -d /content/drive/'My Drive'/${zip_foldername} ]];
then
  cp -r /content/drive/'My Drive'/${zip_foldername} ./

  for (( i = 0; i < 4; i++ ));
  do
    unzip ./${zip_foldername}/from_camera_${i}/zipped_.zip -d ../
    rm -rf ${zip_foldername}/from_camera_${i}/zipped_.zip
  done
else
  echo Folder shoud have only .zip files
fi

In [0]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [0]:
# Pedestrian vs Vehicle

In [0]:
training_loop(
                source_folder="/content/train_0003/content/Data_folder_train_0003_tracks/",
                device=device,
                adam_parameters={"lr": 0.001, "betas" : [0.9, 0.999]},
                shelduler_parameters={"step_size": 20, "gamma": 0.5},
                pointnet_feature_transform=True,
                save_model_to="/content/drive/My Drive/PointNet/descriptors_info/TPvsVehicle",
                epoch_number=30,
                min_pts=30           
             )

In [0]:
# TP vs FP

In [0]:
os.makedirs("/content/TPvsFP_0003/")

In [0]:
!cp -r /content/train_0003/content/Data_folder_train_0003_tracks/TYPE_PEDESTRIAN /content/TPvsFP_0003/

In [0]:
project_utils.merge_fp(fp_dir="/content/content", dest_path="/content/TPvsFP_0003/TYPE_FP_PEDESTRIAN")

In [0]:
training_loop(
                source_folder="/content/TPvsFP_0003/",
                device=device,
                adam_parameters={"lr": 0.001, "betas" : [0.9, 0.999]},
                shelduler_parameters={"step_size": 20, "gamma": 0.5},
                pointnet_feature_transform=True,
                save_model_to="/content/drive/My Drive/PointNet/descriptors_info/TPvsFP/",
                epoch_number=40,
                min_pts=30           
             )

In [0]:
# Track-based model

In [0]:
#os.makedirs("/content/track_based_0003/TYPE_PEDESTRIAN")
#!cp -r /content/train_0003/content/Data_folder_train_0003_tracks/TYPE_PEDESTRIAN /content/TPvsFP_0003/TYPE_PEDESTRIAN

In [0]:
# Need metadata to create TP track
!cp /content/drive/'My Drive'/PointNet/metadata_train_0003_tracks/metadata_TYPE_PEDESTRIAN.csv /content/

In [0]:
project_utils.split_to_tracks(
                               metadata_path="/content/metadata_TYPE_PEDESTRIAN.csv", 
                               source_path="/content/TPvsFP_0003/TYPE_PEDESTRIAN/",
                               dest_path="/content/track_based_0003/"
                             )

In [0]:
#os.mkdir("/content/drive/My Drive/PointNet/descriptors_info/TrackBased/")

training_loop(
                source_folder="/content/track_based_0003/",
                cls_num = len(os.listdir("/content/track_based_0003/")),
                device=device,
                adam_parameters={"lr": 0.001, "betas" : [0.9, 0.999]},
                shelduler_parameters={"step_size": 20, "gamma": 0.5},
                pointnet_feature_transform=True,
                save_model_to="/content/drive/My Drive/PointNet/descriptors_info/TrackBased/",
                epoch_number=40,
                min_pts=30,
                validation_frac=0.0,
                val_step=None           
             )

In [5]:
# Get GT pointclouds and metadata for track validation

!cp -r /content/drive/'My Drive'/PointNet/zip_tracks/ /content/zip_tracks/
!unzip /content/zip_tracks/TYPE_PEDESTRIAN.zip -d /content/unzipped/

!cp /content/drive/'My Drive'/PointNet/metadata_train_0001_dataset/metadata_TYPE_PEDESTRIAN.csv ../

Streaming output truncated to the last 5000 lines.
  inflating: /content/unzipped/TYPE_PEDESTRIAN/00023068.npy  
  inflating: /content/unzipped/TYPE_PEDESTRIAN/00023069.npy  
  inflating: /content/unzipped/TYPE_PEDESTRIAN/00023070.npy  
  inflating: /content/unzipped/TYPE_PEDESTRIAN/00023071.npy  
  inflating: /content/unzipped/TYPE_PEDESTRIAN/00023072.npy  
  inflating: /content/unzipped/TYPE_PEDESTRIAN/00023073.npy  
  inflating: /content/unzipped/TYPE_PEDESTRIAN/00023074.npy  
  inflating: /content/unzipped/TYPE_PEDESTRIAN/00023075.npy  
  inflating: /content/unzipped/TYPE_PEDESTRIAN/00023076.npy  
  inflating: /content/unzipped/TYPE_PEDESTRIAN/00023077.npy  
  inflating: /content/unzipped/TYPE_PEDESTRIAN/00023078.npy  
  inflating: /content/unzipped/TYPE_PEDESTRIAN/00023079.npy  
  inflating: /content/unzipped/TYPE_PEDESTRIAN/00023080.npy  
  inflating: /content/unzipped/TYPE_PEDESTRIAN/00023081.npy  
  inflating: /content/unzipped/TYPE_PEDESTRIAN/00023082.npy  
  inflating: /conte

In [0]:
# Hyperparameters

manual_seed = 32424242424232                # seed for rng
cls_num = 2                                 # number of classes, should match that number in dataloader 
pointnet_feature_transform = True           # Use T-Net block for inner features transformation


In [0]:
torch.manual_seed(manual_seed)

track_loader = track_dataloader("/content/unzipped", min_pts=30, batch_size=32, num_of_workers=0, 
                                cls_to_filter=['TYPE_PEDESTRIAN'])

In [14]:
# TP vs Vehicle tracks
model_checkpoint = "/content/drive/My Drive/PointNet/descriptors_info/TPvsVehicle/cls_model_new_29.pth"

point_net_cls = PointNetCls(k=cls_num, feature_transform=pointnet_feature_transform)
point_net_cls.load_state_dict(torch.load(model_checkpoint))
point_net_cls.to(device)

tp_vs_vehicle_track_matching_cascade = MatchingCascade(track_loader, 
                                                        point_net_cls, 
                                                        "/content/metadata_TYPE_PEDESTRIAN.csv")
tp_vs_vehicle_track_matching_cascade.get_descriptors(dest_path="/content/descriptors_tp_vs_vehicle")

100%|██████████| 878/878 [00:16<00:00, 52.53it/s]


In [0]:
tp_vs_vehicle_track_matching_cascade.start_processing(thresh=0.2, 
                                                      min_pts_num=75, 
                                                      take_last_n=20, 
                                                      agg_function=np.min
                                                      )

In [0]:
# Track view

tp_vs_vehicle_track_matching_cascade.track_dict_ids[tp_vs_vehicle_track_matching_cascade.all_segment_names[6]]

In [247]:
# Track metric #1: Ratio of track mode count to track length (second value is a track length)

tp_vs_vehicle_track_matching_cascade.track_metrics[tp_vs_vehicle_track_matching_cascade.all_segment_names[15]]

[(0.5617977528089888, 3, 89),
 (0.4953271028037383, 4, 107),
 (0.48148148148148145, 4, 108),
 (0.8, 3, 45)]

In [241]:
# Track metric #2: Sum of id switches for segment.

tp_vs_vehicle_track_matching_cascade.re_id_metric()
tp_vs_vehicle_track_matching_cascade.re_id

{'segment-15857303257471811288_1840_000_1860_000_with_camera_labels.tfrecord': 3466,
 'segment-16102220208346880_1420_000_1440_000_with_camera_labels.tfrecord': 17,
 'segment-16652690380969095006_2580_000_2600_000_with_camera_labels.tfrecord': 539,
 'segment-18111897798871103675_320_000_340_000_with_camera_labels.tfrecord': 85,
 'segment-18244334282518155052_2360_000_2380_000_with_camera_labels.tfrecord': 690,
 'segment-18380281348728758158_4820_000_4840_000_with_camera_labels.tfrecord': 347,
 'segment-200287570390499785_2102_000_2122_000_with_camera_labels.tfrecord': 155,
 'segment-2064489349728221803_3060_000_3080_000_with_camera_labels.tfrecord': 0,
 'segment-2607999228439188545_2960_000_2980_000_with_camera_labels.tfrecord': 0,
 'segment-2739239662326039445_5890_320_5910_320_with_camera_labels.tfrecord': 64,
 'segment-2961247865039433386_920_000_940_000_with_camera_labels.tfrecord': 0,
 'segment-33101359476901423_6720_910_6740_910_with_camera_labels.tfrecord': 40,
 'segment-3375636

In [0]:
track_sp = TrackScatterPlotter(track_dict=tp_vs_vehicle_track_matching_cascade.track_dict, 
                               metadata=tp_vs_vehicle_track_matching_cascade.filtered_metadata, 
                               source_path="/content/unzipped/TYPE_PEDESTRIAN")
track_gen = track_sp.visualize_track_generator(tp_vs_vehicle_track_matching_cascade.all_segment_names[3], 0)

In [0]:
next(track_gen)

In [32]:
# TPvsFP tracks

model_checkpoint = "/content/drive/My Drive/PointNet/descriptors_info/TPvsFP/cls_model_new_39.pth"

point_net_cls = PointNetCls(k=cls_num, feature_transform=pointnet_feature_transform)
point_net_cls.load_state_dict(torch.load(model_checkpoint))
point_net_cls.to(device)

tp_vs_fp_track_matching_cascade = MatchingCascade(track_loader, 
                                                        point_net_cls, 
                                                        "/content/metadata_TYPE_PEDESTRIAN.csv")
tp_vs_fp_track_matching_cascade.get_descriptors(dest_path="/content/descriptors_tp_vs_fp")

100%|██████████| 878/878 [00:16<00:00, 53.72it/s]


In [0]:
tp_vs_fp_track_matching_cascade.start_processing(thresh=0.2, 
                                                      min_pts_num=75, 
                                                      take_last_n=20, 
                                                      agg_function=np.min)

In [0]:
# Track view

tp_vs_fp_track_matching_cascade.track_dict_ids[tp_vs_fp_track_matching_cascade.all_segment_names[6]]

In [242]:
# Track metrics

tp_vs_fp_track_matching_cascade.track_metrics[tp_vs_fp_track_matching_cascade.all_segment_names[15]]

[(0.5306122448979592, 3, 98), (0.7575757575757576, 3, 132), (0.8, 4, 115)]

In [0]:
# Track metric #2: Sum of id switches for segment.

tp_vs_fp_track_matching_cascade.re_id_metric()
tp_vs_fp_track_matching_cascade.re_id

In [0]:
track_sp = TrackScatterPlotter(track_dict=tp_vs_fp_track_matching_cascade.track_dict, 
                          metadata=tp_vs_fp_track_matching_cascade.filtered_metadata, 
                          source_path="/content/unzipped/TYPE_PEDESTRIAN")
track_gen = track_sp.visualize_track_generator(tp_vs_fp_track_matching_cascade.all_segment_names[3], 0)

In [39]:
# Track based classification tracks

model_checkpoint = "/content/drive/My Drive/PointNet/descriptors_info/TrackBased/cls_model_new_39.pth"

point_net_cls = PointNetCls(k=827, feature_transform=pointnet_feature_transform)
point_net_cls.load_state_dict(torch.load(model_checkpoint))
point_net_cls.to(device)

track_based_matching_cascade = MatchingCascade(track_loader, 
                                                        point_net_cls, 
                                                        "/content/metadata_TYPE_PEDESTRIAN.csv")
track_based_matching_cascade.get_descriptors(dest_path="/content/descriptors_track_based")

100%|██████████| 878/878 [00:16<00:00, 53.43it/s]


In [0]:
track_based_matching_cascade.start_processing(thresh=0.2, 
                                                      min_pts_num=75, 
                                                      take_last_n=20, 
                                                      agg_function=np.min)

In [0]:
# Track view

track_based_matching_cascade.track_dict_ids[track_based_matching_cascade.all_segment_names[15]]

In [0]:
# Track metrics

track_based_matching_cascade.track_metrics[track_based_matching_cascade.all_segment_names[15]]

In [252]:
# Track metric #2: Sum of id switches for segment.

track_based_matching_cascade.re_id_metric()
track_based_matching_cascade.re_id

{'segment-15857303257471811288_1840_000_1860_000_with_camera_labels.tfrecord': 2832,
 'segment-16102220208346880_1420_000_1440_000_with_camera_labels.tfrecord': 11,
 'segment-16652690380969095006_2580_000_2600_000_with_camera_labels.tfrecord': 372,
 'segment-18111897798871103675_320_000_340_000_with_camera_labels.tfrecord': 51,
 'segment-18244334282518155052_2360_000_2380_000_with_camera_labels.tfrecord': 527,
 'segment-18380281348728758158_4820_000_4840_000_with_camera_labels.tfrecord': 236,
 'segment-200287570390499785_2102_000_2122_000_with_camera_labels.tfrecord': 28,
 'segment-2064489349728221803_3060_000_3080_000_with_camera_labels.tfrecord': 0,
 'segment-2607999228439188545_2960_000_2980_000_with_camera_labels.tfrecord': 0,
 'segment-2739239662326039445_5890_320_5910_320_with_camera_labels.tfrecord': 52,
 'segment-2961247865039433386_920_000_940_000_with_camera_labels.tfrecord': 0,
 'segment-33101359476901423_6720_910_6740_910_with_camera_labels.tfrecord': 22,
 'segment-33756369

In [0]:
track_sp = TrackScatterPlotter(track_dict=track_based_matching_cascade.track_dict, 
                               metadata=track_based_matching_cascade.filtered_metadata, 
                               source_path="/content/unzipped/TYPE_PEDESTRIAN")
track_gen = track_sp.visualize_track_generator(track_based_matching_cascade.all_segment_names[3], 0)

In [0]:
next(track_gen)

16179 JpYKZOtPQGQOgSSfNX418Q
